In [26]:
import pandas as pd
import json
import numpy as np
import os
from google.cloud import storage
from io import BytesIO
from pypdf import PdfReader  # You may need to install pypdf if you haven't already
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain
#from langchain.llms import OpenAI
import vertexai
from vertexai.generative_models import GenerativeModel, Part
from langchain_google_vertexai import ChatVertexAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import logging

In [27]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'xenon-heading-430209-e4-7582f64a7330.json' # Rellenar con el .json correspondiente
logging.basicConfig(filename='error_log.txt', level=logging.ERROR, format='%(asctime)s %(message)s')


In [28]:
def leer_blob_en_memoria(bucket_name, blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    pdf_content = blob.download_as_bytes()
    return pdf_content

def extraer_texto_de_pdf_bytes(pdf_bytes):
    pdf_file = BytesIO(pdf_bytes)
    reader = PdfReader(pdf_file)
    text = ''
    for page in reader.pages:
        text += page.extract_text() + '\n'
    return text

def procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name):
    pdf_bytes = leer_blob_en_memoria(bucket_name, blob_name)
    content = extraer_texto_de_pdf_bytes(pdf_bytes)
    return content

def listar_pdfs(bucket_name):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blobs = bucket.list_blobs()

    # Filtra los archivos PDF y guarda sus URLs completas en la lista
    listado_pdfs = [f"gs://{bucket_name}/{blob.name}" for blob in blobs if blob.name.endswith('.pdf') and not blob.name.startswith('resoluciones/')]
    return listado_pdfs

def dividir_lista_pdfs(listado_pdfs):
    total = len(listado_pdfs)
    tamaño_parte = total // 5
    
    parte1 = listado_pdfs[:tamaño_parte]
    parte2 = listado_pdfs[tamaño_parte:2*tamaño_parte]
    parte3 = listado_pdfs[2*tamaño_parte:3*tamaño_parte]
    parte4 = listado_pdfs[3*tamaño_parte:4*tamaño_parte]
    parte5 = listado_pdfs[4*tamaño_parte:]
    
    return parte1, parte2, parte3, parte4, parte5

In [29]:
bucket_name = 'tfm_javi'

In [32]:
# Definir los esquemas de respuesta
response_schemas = [
    ResponseSchema(name="numero_expediente", description="Número de Expediente, ejemplo SD2023/0000046"),
    ResponseSchema(name="resolucion", description="Una de las siguientes opciones: negada_con_oposicion, negada_sin_oposicion, aprobada_sin_oposicion, aprobada_con_oposición"),
    ResponseSchema(name="numero_de_resolución", description="Número entero de la resolución, ejemplo 2195"),
    ResponseSchema(name="denominacion", description="Nombre de la empresa que solicita el registro de la marca"),
    ResponseSchema(name="vigencia", description="Fecha en que expira la vigencia del registro, o texto si es negada o vencida"),
    ResponseSchema(name="titular", description="Titular de la marca que intenta registrar"),
    ResponseSchema(name="clase", description="Número o lista de números de la Clasificación Internacional de Niza, por ejemplo [42, 35, 27]"),
    ResponseSchema(name="gaceta", description="Número de la gaceta de Propiedad Industrial donde se publica"),
    ResponseSchema(name="tipo", description="Tipo de registro que se intenta realizar, por ejemplo Mixta, Nominativa, Figurativa"),
    ResponseSchema(name="fecha_solicitud", description="Fecha de presentación de la solicitud"),
    ResponseSchema(name="fecha_resolucion", description="Fecha de resolución"),
    ResponseSchema(name="nombre_opositor", description="Nombre de la empresa que se opone a la publicación"),
    ResponseSchema(name="signo_opositor_opositores", description="Signo o signos de los opositores en conflicto"),
    ResponseSchema(name="argumento_oposición", description="Argumentos en los que se basa para oponerse al registro y artículos en los que se apoya"),
    ResponseSchema(name="explicacion_argumentos_oposicion", description="Breve resumen y explicación de los argumentos de la oposición"),
    ResponseSchema(name="resolucion_organismo", description="Resolución del organismo competente, por ejemplo: 'DENIEGA el registro de la marca PAPELES LA FAVORITA (Mixta)'")
]


In [33]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
# Escapar las llaves para evitar la interpretación de variables
format_instructions = format_instructions.replace("{", "{{").replace("}", "}}")

In [34]:
system_prompt = """
Eres un Experto abogado Colombiano en analizar resoluciones del SIC (Superintendencia de Industria y Comercio de Colombia),en el ámbito de registro de marcas y lemas.
quiero que extraigas el numero de Expediente, la resolución del conflicto, el numero de la resolución, el nombre de la marca que intenta registrarse, el titular que intenta registrar la marca, el numero de clase que intenta registrar, 
el numero de la gaceta en que ha sido publicada, la fecha de solicitud de registro, nombre de la empresa opositora, el titular de la empresa que se opone si aparece, y los argumentos de derecho en los que se apoya el opositor.
"""
human_prompt = f"""Extrae la información indicada en DATOS a partir del TEXTO de la resolución

TEXT
---
\n\n{{contenido_pdf}}
---

DATOS
{format_instructions}
"""


In [35]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template(human_prompt),
])

In [36]:
# Define tu LLM 
llm = ChatVertexAI(
    model="gemini-1.5-pro-001"
      #verificar la relacion megas tokens  
)

In [37]:
chain = LLMChain(llm=llm, prompt=prompt, output_parser=output_parser)


In [40]:
# Obtener la lista de PDFs
listado_pdfs = listar_pdfs(bucket_name)

In [ ]:
len(listado_pdfs)

In [42]:
parte1, parte2, parte3, parte4, parte5 = dividir_lista_pdfs(listado_pdfs)

In [ ]:
parte5

In [20]:
# Inicializar una lista para almacenar los resultados
resultados = []


In [ ]:
 for pdf_uri in parte5:
     blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
     contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
      # Verificar si el contenido del PDF no está vacío
     if contenido_pdf.strip():
         try:
              # Invocar la cadena con el contenido del PDF
             res = chain.invoke({"contenido_pdf": contenido_pdf})
              #Agregar el resultado a la lista de resultados
             resultados.append(res)
              #Imprimir el resultado
             print(f"Resultado para {blob_name}:\n{res}\n")
         except Exception as e:
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
     else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

## Errores de analisis
realizamos en la terminal cat error_log.txt que nos da el resultado de:

error al procesar resoluciones/NCO-SD2022-0073142.pdf

error al procesar resoluciones/NCO-SD2022-0113402.pdf

error al procesar resoluciones/SD2022-0017210.pdf

vamos  a realizar un intento individual con esos documentos

In [60]:
listado_pdfs_error = (
    #f"gs://{bucket_name}/NCO-SD2018-0088625.pdf",
    #f"gs://{bucket_name}/NCO-SD2018-0098791.pdf",
    #f"gs://{bucket_name}/NCO-SD2019-0020998.pdf",
    #f"gs://{bucket_name}/NCO-SD2019-0025443.pdf",
    f"gs://{bucket_name}/NCO-SD2019-0077987.pdf",
    f"gs://{bucket_name}/NCO-SD2019-0086865.pdf",
    f"gs://{bucket_name}/NSO-SD2020-0067729.pdf",
    f"gs://{bucket_name}/NCO-SD2020-0070008.pdf",
    f"gs://{bucket_name}/NSO-SD2020-0072423.pdf",
    f"gs://{bucket_name}/NCO-SD2020-0093067.pdf",
    f"gs://{bucket_name}/NCO-SD2020-0107440.pdf",
    f"gs://{bucket_name}/NCO-SD2021-0001032.pdf",
    f"gs://{bucket_name}/NCO-SD2021-0011099.pdf",
    f"gs://{bucket_name}/NCO-SD2021-0024718.pdf",
    f"gs://{bucket_name}/NCO-SD2021-0045759.pdf",
    f"gs://{bucket_name}/NCO-SD2021-0059591.pdf",
    f"gs://{bucket_name}/NCO-SD2021-0066429.pdf",
    f"gs://{bucket_name}/NCO-SD2021-0085391.pdf",
    f"gs://{bucket_name}/NCO-SD2022-0011807.pdf",
    f"gs://{bucket_name}/NCO-SD2022-0014356.pdf",
    f"gs://{bucket_name}/NCO-SD2022-0028602.pdf",
    f"gs://{bucket_name}/NCO-SD2022-0038932.pdf",
    f"gs://{bucket_name}/NCO-SD2022-0127271.pdf",
)

In [ ]:
len(listado_pdfs_error)

In [ ]:
listado_pdfs_error

In [ ]:
 for pdf_uri in listado_pdfs_error:
     blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
     contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
      # Verificar si el contenido del PDF no está vacío
     if contenido_pdf.strip():
         try:
              # Invocar la cadena con el contenido del PDF
             res = chain.invoke({"contenido_pdf": contenido_pdf})
              #Agregar el resultado a la lista de resultados
             resultados.append(res)
              #Imprimir el resultado
             print(f"Resultado para {blob_name}:\n{res}\n")
         except Exception as e:
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
     else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

Quiero volver a intentar este registro CSO-SD2022-0105220.pdf

Parece que esta vez si que ha procesado bien los tres archivos que diéron error, vamos a construir un df con los resultados

In [64]:
df_resultados = pd.DataFrame(resultados)

In [ ]:
df_resultados

In [66]:
pd.set_option('display.max_columns', None)

In [67]:
df_ai_annotations_full_5 = pd.DataFrame.from_records(df_resultados['text'])

In [ ]:
df_ai_annotations_full_5

In [ ]:
df_ai_annotations_full_1.info()

In [ ]:
df_ai_annotations["argumento_oposición"]

In [69]:
df_ai_annotations_full_5.to_csv("datasetia_full_5.csv")

In [6]:
df = pd.read_csv("datasetia.csv")

In [7]:
df = df.drop(columns=['Unnamed: 0'])

In [ ]:
df

***JDS:*** *veo haciendo una exploración que XING FA es una marca que al parecer ya esta registrada, y que la SIC ha entrado de oficio en un caso (expediente SD2022/0011834) porque entra en conflicto con esa marca, hago una exploración muy basica a ver si es uno de los casos que ya hemos procesado y aparentemete no, es posible que si que este pero habria que normalizarlo a minusculas etc, pero tambien es muy posible que no este al ser solo 419 elementos del total, habria que hacer un merge con la bbdd del sql que nos dieron de sysmarck*

In [ ]:
df[df["denominacion"].str.contains("XING FA", case=False, na=False)]